In [1]:
DATE = "2021-06-15"
TASK = "100bp-0.1-error-rate"
DIR = "$(DATE)-$(TASK)"
DIR = mkpath("$(homedir())/$(DIR)")

"/Users/cameronprybol/2021-06-15-100bp-0.1-error-rate"

In [78]:
pkgs = [
"LightGraphs",
"MetaGraphs",
"BioSequences",
"uCSV",
"DataFrames",
"FASTX",
"HTTP",
"CodecZlib",
"DataStructures",
"Revise",
"ProgressMeter",
"BenchmarkTools",
"Random",
"Images",
"ImageTransformations",
"StatsBase",
"SparseArrays"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

In [3]:
# # 11, 17 is too small
# for k in [13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     # if !isfile(outgraph)
#     accessions = ncbi_staph_phage_metadata[!, "Accession"]
#     fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#     KMER_TYPE = BioSequences.BigDNAMer{k}
#     @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)

#     @info "simplifying kmer graph"
#     @info "resolving untigs..."
#     @time untigs = Mycelia.resolve_untigs(kmer_graph)

#     @info "determining untig orientations..."
#     @time oriented_untigs = Mycelia.determine_oriented_untigs(kmer_graph, untigs)

#     simplified_graph = MetaGraphs.MetaDiGraph(length(oriented_untigs))
#     MetaGraphs.set_prop!(simplified_graph, :k, kmer_graph.gprops[:k])
#     @info "initializing graph node metadata"
#     ProgressMeter.@showprogress for (vertex, untig) in enumerate(oriented_untigs)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :sequence, untig.sequence)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :path, untig.path)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :orientations, untig.orientations)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :evidence, untig.evidence)
#     end

#     # determine oriented edges of simplified graph
#     simplified_untigs = Vector{Pair{Pair{Int64,Bool},Pair{Int64,Bool}}}(undef, length(LightGraphs.vertices(simplified_graph)))
#     @info "creating simplified unitgs to help resolve connections"
#     # use a pre-allocated array here to speed up
#     ProgressMeter.@showprogress for vertex in LightGraphs.vertices(simplified_graph)
#         in_kmer = simplified_graph.vprops[vertex][:path][1] => simplified_graph.vprops[vertex][:orientations][1]
#         out_kmer = simplified_graph.vprops[vertex][:path][end] => simplified_graph.vprops[vertex][:orientations][end]
#     #     @show vertex, in_kmer, out_kmer
#         simplified_untigs[vertex] = in_kmer => out_kmer
#     #     push!(simplified_untigs, )
#     end

#     # make a dictionary mapping endcap to oriented_untig index

#     end_mer_map = Dict()
#     ProgressMeter.@showprogress for (i, oriented_untig) in enumerate(oriented_untigs)
#         end_mer_map[first(oriented_untig.path)] = i
#         end_mer_map[last(oriented_untig.path)] = i
#     end

#     ProgressMeter.@showprogress for (untig_index, oriented_untig) in enumerate(oriented_untigs)
#     #     @show untig_index
#         true_in_overlap = oriented_untig.sequence[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == true_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == true_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         true_out_overlap = oriented_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_in_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == false_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == false_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_out_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end

#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end
#     end

#     # ^ > 1 day down to 34 seconds!
#     Mycelia.graph_to_gfa(simplified_graph, outgraph)
# end

In [4]:
# set a random seed
seed = Random.seed!(0)

MersenneTwister(0)

In [5]:
# randomly generate a dna sequence of 100bp
genome = BioSequences.randdnaseq(seed, 100)

100nt DNA Sequence:
AAGGGTGCGGTCTAGGTGCACTGCTTATGGTCCCCGACA…TCTACCTCTTTGTTGAAACGTATTCTTCTCTTAACACCT

In [6]:
# define error rate
error_rate = 0.1

0.1

In [7]:
# generate 100x coverage fastq file
# put accuracy rate into fastq file
coverage = 100
fastq_file = "$(DIR)/$(DATE)-$(TASK).fastq"
open(fastq_file, "w") do io
    fastq_writer = FASTX.FASTQ.Writer(io)
    for i in 1:coverage
        observed_sequence = Mycelia.observe(genome, error_rate=error_rate)
        q = -10 * log10(error_rate)
        quality_scores = fill(q, length(observed_sequence))
        fastq_record = FASTX.FASTQ.Record("i", observed_sequence, quality_scores)
        write(fastq_writer, fastq_record)
    end
end

In [8]:
# build weighted kmer graph
# at some point where we may want to record weights as integers rather than lists of evidence, but first confirm the algorithm

In [9]:
k = Int(1/error_rate + 1)

11

In [10]:
kmer_type = BioSequences.BigDNAMer{k}

BioSequences.BigDNAMer{11} (alias for BioSequences.BigMer{BioSequences.DNAAlphabet{2}, 11})

In [11]:
kmer_graph = Mycelia.fastx_to_kmer_graph(kmer_type, fastq_file)

┌ Info: assessing kmers
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1877
┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1891


{4710, 10148} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [12]:
# visualize
gfa_file = fastq_file * ".k-$k.gfa"
Mycelia.graph_to_gfa(kmer_graph, gfa_file)

"/Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa"

In [27]:
run(`/Applications/Bandage.app/Contents/MacOS/Bandage image $(gfa_file) $(gfa_file).svg --depwidth 1 --deppower 1`)
# --nodewidth <float> Average node width (0.5 to 1000, default: 5)
# --depwidth <float>  Depth effect on width (0 to 1, default: 0.5)
# --deppower <float>  Power of depth effect on width (0 to 1, default: 0.5)

Process(`/Applications/Bandage.app/Contents/MacOS/Bandage image /Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa /Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa.svg --depwidth 1 --deppower 1`, ProcessExited(0))

In [50]:
x = "<img src=../../../../2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa.svg>"
display("text/html", x)
# display("image/svg+xml", read("$(gfa_file).svg", String))

In [51]:
fastq_io = FASTX.FASTQ.Reader(open(fastq_file))

FASTX.FASTQ.Reader{TranscodingStreams.NoopStream{IOStream}}(BioGenerics.Automa.State{TranscodingStreams.NoopStream{IOStream}}(TranscodingStreams.NoopStream{IOStream}(<mode=idle>), 1, 1, false), nothing)

In [53]:
fastq_read = first(fastq_io)

FASTX.FASTQ.Record:
   identifier: i
  description: <missing>
     sequence: AAGGGTGCGTCAAGGTGTACTGCTATGGTCCCCGACAGGACCGTGCGGTGATTTTCTGAACTACCACTTTGTTGAAACGTATTCTTCTCTTAACACCT
      quality: UInt8[0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a  …  0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a]

In [57]:
# for q in FASTX.quality(fastq_read)
#     q = -10 * log10(error_rate)
#     likelihood_error = 10^(q/(-10))
#     display(likelihood_error)
# end

In [ ]:
state_likelihoods = 

In [64]:
kmer_graph.vprops[1]

Dict{Symbol, Any} with 2 entries:
  :evidence => Set(NamedTuple{(:record, :index, :orientation), Tuple{String, In…
  :kmer     => AAAAAGAGGTA

In [66]:
kmers = [kmer_graph.vprops[v][:kmer] for v in LightGraphs.vertices(kmer_graph)]

4710-element Vector{BioSequences.BigDNAMer{11}}:
 AAAAAGAGGTA
 AAAAAGCAGGT
 AAAAAGGAGGT
 AAAAAGGAGTA
 AAAAAGGTAGA
 AAAAAGGTAGT
 AAAAATACGTT
 AAAACAAGAGG
 AAAACGTATTC
 AAAACGTTTCA
 AAAAGAGACCT
 AAAAGAGGTAA
 AAAAGCAGGTA
 ⋮
 TTGCGTTACAA
 TTGGTATGAAA
 TTGTCATGAAA
 TTGTGTTGAAA
 TTGTTTTGAAA
 TTTAAACAAAA
 TTTCAAACAAA
 TTTCAACAAAA
 TTTCAACCAAA
 TTTCAACGAAA
 TTTCACACAAA
 TTTCCACAAAA

In [67]:
kmer_counts = [length(kmer_graph.vprops[v][:evidence]) for v in LightGraphs.vertices(kmer_graph)]

4710-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 3
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 2
 2
 1
 1
 1
 2

In [69]:
StatsBase.describe(kmer_counts)

Summary Stats:
Length:         4710
Missing Count:  0
Mean:           1.458174
Minimum:        1.000000
1st Quartile:   1.000000
Median:         1.000000
3rd Quartile:   1.000000
Maximum:        17.000000
Type:           Int64


In [70]:
kmer_likelihoods = kmer_counts ./ sum(kmer_counts)

4710-element Vector{Float64}:
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00043680838672102506
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 ⋮
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00029120559114735004
 0.00029120559114735004
 0.00014560279557367502
 0.00014560279557367502
 0.00014560279557367502
 0.00029120559114735004

In [71]:
StatsBase.describe(kmer_likelihoods)

Summary Stats:
Length:         4710
Missing Count:  0
Mean:           0.000212
Minimum:        0.000146
1st Quartile:   0.000146
Median:         0.000146
3rd Quartile:   0.000146
Maximum:        0.002475
Type:           Float64


In [ ]:
# evaluate likelihood of first read

In [73]:
kmer_type = BioSequences.BigDNAMer{kmer_graph.gprops[:k]}

BioSequences.BigDNAMer{11} (alias for BioSequences.BigMer{BioSequences.DNAAlphabet{2}, 11})

In [77]:
kmer_index, fw, bw = first(collect(BioSequences.each(kmer_type, FASTX.sequence(fastq_read))))

Mer iteration result:
Position: 1
Forward: AAGGGTGCGTC
Backward: GACGCACCCTT


In [121]:
transition_likelihoods = Dict(
    true => SparseArrays.spzeros(K, K),
    false => SparseArrays.spzeros(K, K)
)

for edge in collect(LightGraphs.edges(kmer_graph))
    weight = length(kmer_graph.eprops[edge][:evidence])
    for o in kmer_graph.eprops[edge][:orientations]
        transition_likelihoods[o.source_orientation][edge.src, edge.dst] = weight
    end
end

for source_orientation in (true, false)
    for src in 1:K
        transition_weights = transition_likelihoods[source_orientation][src, :]
        total_weight = sum(transition_weights)
        dsts, vals = SparseArrays.findnz(transition_weights)
        for (dst, val) in zip(dsts, vals) 
            transition_likelihoods[source_orientation][src, dst] = val / total_weight
        end
        normalized_probability = sum(transition_likelihoods[source_orientation][src, :])
        @assert isapprox(normalized_probability, 0) || isapprox(normalized_probability, 1)
    end
end

In [ ]:
# evaluate probability of each state

In [ ]:
# backtrack most likely path

In [ ]:
# write sequence to output fastq

In [ ]:
# for k in [11, 13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     if !isfile(outgraph)
#         accessions = ncbi_staph_phage_metadata[!, "Accession"]
#         fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#         KMER_TYPE = BioSequences.BigDNAMer{k}
#         @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)
#         @time simplified_graph = Mycelia.simplify_kmer_graph(kmer_graph)
# #         ^ > 1 day down to 34 seconds!
#         Mycelia.graph_to_gfa(simplified_graph, outgraph)
#     end
# end